#  作業

Hint: 人物原始邊框座標 (60, 40), (420, 510)

請根據 Lena 圖做以下處理

- 對明亮度做直方圖均衡處理
- 水平鏡像 + 縮放處理 (0.5 倍)
- 畫出人物矩形邊框

In [1]:
import cv2
import numpy as np

img = cv2.imread('images/lena.png')

## Hint: 矩形

In [3]:
img_rect = img.copy()
cv2.rectangle(img_rect, (60, 40), (420, 510), (0, 0, 255), 3)

cv2.imshow('image', img_rect)

# 按下任意鍵則關閉所有視窗
cv2.waitKey(0)
cv2.destroyAllWindows()

## Hint: 線

In [4]:
img_line = img.copy()
cv2.line(img_line, (60, 40), (420, 510), (0, 0, 255), 3)

cv2.imshow('image', img_line)

# 按下任意鍵則關閉所有視窗
cv2.waitKey(0)
cv2.destroyAllWindows()


## Hint: 文字

In [7]:
img_text = img.copy()
cv2.putText(img_text, '(60, 40)', (60, 40), 0, 1, (0, 0, 255), 2)

cv2.imshow('image', img_text)

# 按下任意鍵則關閉所有視窗
cv2.waitKey(0)
cv2.destroyAllWindows()

##  解法二

如果希望得知矩型邊框的位置

1. 顏色的操作 (對明亮度做直方圖均衡)
2. 鏡像可以透過四則運算得知
3. 透過建構 transformation matrix 做縮放
4. 把矩型邊框的點與 transformation matrix 相乘就會得到縮放後的位置
5. 畫圖

得到的圖的結果正確，同時也知道新的矩型邊框座標點

In [4]:
img_hw = img.copy()
point1 = [60, 40]
point2 = [420, 510]

# 直方圖處理 : V
img_hsv = cv2.cvtColor(img_hw, cv2.COLOR_BGR2HSV)
img_hsv[..., 2] = cv2.equalizeHist(img_hsv[..., 2])
img_hw = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)

# 水平翻轉
img_hw = img_hw[:, ::-1, :]

# 透過四則運算計算鏡像後位置
# 確保點的位置一樣是左上跟右下，所以交換鏡像後的 x 座標 (y 座標做水平鏡像後位置不變)
h, w = img_hw.shape[:2]
point1[0] = w - point1[0]
point2[0] = w - point2[0]

# 縮放處理 (0.5 倍)
fx = 0.5
fy = 0.5
img_hw = cv2.resize(img_hw, None, fx=fx, fy=fy)

# 畫矩形 (-50%)
scale_point1 = tuple(np.dot(fx, point1).astype('uint8'))
scale_point2 = tuple(np.dot(fy, point2).astype('uint8'))
cv2.rectangle(img_hw, scale_point1, scale_point2, (0, 0, 255), 2)

print('origin point1={}, origin point2={}'.format(point1, point2))
print('resize point1={}, resize point2={}'.format(scale_point1, scale_point2))

cv2.imshow('image', img_hw)

# 按下任意鍵則關閉所有視窗
cv2.waitKey(0)
cv2.destroyAllWindows()

origin point1=[452, 40], origin point2=[92, 510]
resize point1=(226, 20), resize point2=(46, 255)


##  解法三 (optional)

把矩型邊框用遮罩的方式呈現，使用同樣處理圖片的方式處理遮罩
最後再找遮罩的左上跟右下的點的位置

這邊會用到許多沒提過的東西，所以當作 optional